In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
import pandas as pd
import numpy as np

normal_df = pd.read_csv("/content/Healthcare Providers.csv")
anomaly_df = pd.read_csv("/content/Healthcare Providers.csv")
normal_df.head()

,index,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Middle Initial of the Provider,Credentials of the Provider,Gender of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,Street Address 2 of the Provider,...,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
0,8774979,1891106191,UPADHYAYULA,SATYASREE,NaN,M.D.,F,I,1402 S GRAND BLVD,FDT 14TH FLOOR,...,99223,"Initial hospital inpatient care, typically 70 ...",N,27,24,27,200.58777778,305.21111111,157.26222222,160.90888889
1,3354385,1346202256,JONES,WENDY,P,M.D.,F,I,2950 VILLAGE DR,NaN,...,G0202,"Screening mammography, bilateral (2-view study...",N,175,175,175,123.73,548.8,118.83,135.31525714
2,3001884,1306820956,DUROCHER,RICHARD,W,DPM,M,I,20 WASHINGTON AVE,STE 212,...,99348,"Established patient home visit, typically 25 m...",N,32,13,32,90.65,155,64.4396875,60.5959375
3,7594822,1770523540,FULLARD,JASPER,NaN,MD,M,I,5746 N BROADWAY ST,NaN,...,81002,"Urinalysis, manual test",N,20,18,20,3.5,5,3.43,3.43
4,746159,1073627758,PERROTTI,ANTHONY,E,DO,M,I,875 MILITARY TRL,SUITE 200,...,96372,Injection beneath the skin or into muscle for ...,N,33,24,31,26.52,40,19.539393939,19.057575758


In [3]:
# Data Labeling
CLASS_NAMES = ["Normal", "Anomaly"]
normal_df_copy = normal_df.copy().assign(target=CLASS_NAMES[0])
anomaly_df_copy = anomaly_df.copy().assign(target=CLASS_NAMES[1])
df = pd.concat((normal_df_copy, anomaly_df_copy), ignore_index=True)

In [4]:
# Prepare independent variables and target labels
X = df.drop("target", axis=1, errors="ignore")  # Features
y = df["target"].apply(lambda x: 1 if x == CLASS_NAMES[1] else 0)  # Binary labels'''


In [5]:

# Step 1: Handle Semicolon-Separated Values
for col in X.columns:
    if X[col].astype(str).str.contains(';').any():
        # Split and expand semicolon-separated columns
        split_data = X[col].str.split(';', expand=True)
        split_data = split_data.apply(pd.to_numeric, errors='coerce')  # Convert to numeric
        split_data.columns = [f"{col}_{i}" for i in range(split_data.shape[1])]  # Rename columns
        X = pd.concat([X.drop(columns=[col]), split_data], axis=1)  # Replace original column


In [6]:
# Step 2: Convert Non-Numeric Columns
object_columns = X.select_dtypes(include=['object']).columns
for col in object_columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')  # Convert non-numeric to numeric


In [7]:
# Preprocess the data: handle object columns, encode or drop them if needed
object_columns = X.select_dtypes(include=['object']).columns
if len(object_columns) > 0:
    for col in object_columns:
        X[col] = pd.factorize(X[col])[0]

In [8]:
from sklearn.impute import SimpleImputer
# Step 3: Handle Missing Values
imputer = SimpleImputer(strategy="mean")  # Choose appropriate strategy (mean, median, mode)
X = imputer.fit_transform(X)  # Impute missing values
# Split data into training and testing sets


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['Last Name/Organization Name of the Provider'
 'First Name of the Provider' 'Middle Initial of the Provider'
 'Gender of the Provider' 'Entity Type of the Provider'
 'Street Address 1 of the Provider' 'City of the Provider'
 'State Code of the Provider' 'Country Code of the Provider'
 'Provider Type' 'Medicare Participation Indicator' 'Place of Service'
 'HCPCS Drug Indicator' 'Credentials of the Provider_0'
 'Credentials of the Provider_1' 'HCPCS Description_0'
 'HCPCS Description_1']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [9]:
# Import the necessary function
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.90, random_state=42) # Adjust test_size and random_state as needed

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, test_size=0.4, random_state=42) # Adjust test_size and random_state as needed
print (X_train.shape, X_test.shape)

(12000, 13) (8000, 13)


In [10]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')


Exception ignored in: <function _xla_gc_callback at 0x7f8884edf7f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [15]:
# Assuming 'text_column' is the column with textual data
if 'text_column' in df.columns:
    text_data = df['text_column'].astype(str).tolist()

    # Tokenize text data
    tokens = tokenizer(
        text_data,
        max_length=128,  # Adjust based on memory and dataset
        padding=True,
        truncation=True,
        return_tensors='np'
    )
# Get BERT embeddings
    bert_embeddings = bert_model(tokens['input_ids'])[0][:, 0, :]  # Use [CLS] token embeddings
 # Add BERT embeddings to the dataset
    bert_embeddings = np.array(bert_embeddings)
    X = np.hstack((X, bert_embeddings))  # Combine BERT embeddings with numerical features


IndentationError: unexpected indent (<ipython-input-14-02a92008d376>, line 2)

In [16]:
# Preprocess as before: Scaling and PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)  # Fit on training data
X_test_scaled = scaler.transform(X_test)  # Transform test data


In [17]:
# Feature Extraction (PCA)
n_components = 10  # Adjust PCA components
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)


In [18]:
# Define anomaly data (replace this with actual anomaly data)
anomaly = np.random.rand(200, X_test.shape[1])  # Example anomaly data  # Example anomaly data
# Define anomaly data (replace this with actual anomaly data)
anomaly = np.random.rand(200, X_test.shape[1])  # Example anomaly data
anomaly_scaled = scaler.fit_transform(anomaly)
y_anomaly_true = np.zeros(len(anomaly_scaled))  # Label anomaly data as 0
# else: #Remove this line or the if statement to resolve the unexpected indent
#     print("Error: X_test has no features after processing object columns.")

In [19]:
# Reshape for LSTM-RNN model
time_steps = 10  # Adjust this based on data
trimmed_size_train = (X_pca.shape[0] // time_steps) * time_steps
trimmed_size_test = (X_test_pca.shape[0] // time_steps) * time_steps

X_train_trimmed = X_pca[:trimmed_size_train]
X_test_trimmed = X_test_pca[:trimmed_size_test]


In [20]:
X_train_rnn = np.reshape(X_train_trimmed, (X_train_trimmed.shape[0] // time_steps, time_steps, X_train_trimmed.shape[1]))
X_test_rnn = np.reshape(X_test_trimmed, (X_test_trimmed.shape[0] // time_steps, time_steps, X_test_trimmed.shape[1]))


In [22]:
# Step 1: Define the LSTM-RNN hybrid model
def create_lstm_rnn_hybrid_model(input_shape):
    model = Sequential([
        LSTM(128, input_shape=input_shape, return_sequences=True),
        SimpleRNN(32, return_sequences=True),
        Dense(input_shape[1], activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')  # Mean Squared Error for reconstruction
    return model

# Step 2: Create the model
input_shape = X_train_rnn.shape[1:]  # Shape should be [timesteps, features]
model = create_lstm_rnn_hybrid_model(input_shape)

# Step 3: Train the model
# The model is trained to reconstruct the training data
model.fit(X_train_rnn, X_train_rnn, epochs=100, batch_size=16, validation_split=0.1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 0.5211 - val_loss: 0.0405
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0339 - val_loss: 0.0190
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0148 - val_loss: 0.0127
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 0.0118 - val_loss: 0.0106
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.0088 - val_loss: 0.0100
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.0071 - val_loss: 0.0078
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0073 - val_loss: 0.0073
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0067 - val_loss: 0.0072
Epoch 9/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0056 - val_loss: 0.0051
Epoch 10/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0041 - val_loss: 0.0058
Epoch 11/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0046 - val_loss: 0.0059
Epoch 12/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step

In [28]:
# Step 3: Detect anomalies using reconstruction error
def detect_anomalies_reconstruction_error(model, data, threshold=None):
    # Predict the reconstruction of the input data
    reconstructed_data = model.predict(data)
    # Calculate the reconstruction error (e.g., Mean Squared Error)
    reconstruction_errors = np.mean(np.square(data - reconstructed_data), axis=(1, 2))  # Error across all features
    # Set a threshold based on the 95th percentile of the reconstruction error if not provided
    if threshold is None:
        threshold = np.percentile(reconstruction_errors, 95)

    # Mark data points with reconstruction error greater than threshold as anomalies
    anomalies = reconstruction_errors > threshold
    return anomalies, reconstruction_errors, threshold

In [29]:
# Step 5: Detect anomalies on test data
test_anomalies, test_reconstruction_errors, test_threshold = detect_anomalies_reconstruction_error(model, X_test_rnn)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [31]:
print(f"Shape of anomaly_scaled: {anomaly_scaled.shape}")
print(f"Expected shape: ({anomaly_scaled.shape[0]}, {time_steps}, {X_test_rnn.shape[2]})")


Shape of anomaly_scaled: (200, 13)
Expected shape: (200, 10, 10)


In [32]:
# Ensure anomaly_scaled can be reshaped into the correct dimensions
expected_size = anomaly_scaled.shape[0] * anomaly_scaled.shape[1]
target_size = anomaly_scaled.shape[0] * time_steps * X_test_rnn.shape[2]

if expected_size < target_size:
    # Padding: Add zeros to meet the required size
    padding_size = target_size - expected_size
    anomaly_scaled = np.pad(anomaly_scaled, ((0, 0), (0, padding_size // anomaly_scaled.shape[0])), mode='constant')
elif expected_size > target_size:
    # Trimming: Remove excess elements to match the required size
    anomaly_scaled = anomaly_scaled.flatten()[:target_size].reshape(
        anomaly_scaled.shape[0], time_steps, X_test_rnn.shape[2]
    )
else:
    # Reshape directly if dimensions align
    anomaly_scaled_reshaped = anomaly_scaled.reshape(
        anomaly_scaled.shape[0], time_steps, X_test_rnn.shape[2]
    )


In [33]:
# Step 6: Detect anomalies on anomaly data (if available)
# Ensure anomaly_scaled has the correct shape for the model
anomaly_scaled_reshaped = anomaly_scaled.reshape(anomaly_scaled.shape[0], time_steps, X_test_rnn.shape[2])
anomaly_anomalies, anomaly_reconstruction_errors, anomaly_threshold = detect_anomalies_reconstruction_error(
    model, anomaly_scaled_reshaped, threshold=test_threshold
)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


In [34]:
# Combine ground truth labels and predicted labels
y_true = np.concatenate([np.ones(len(test_anomalies)), np.zeros(len(anomaly_anomalies))])
y_pred = np.concatenate([~test_anomalies, ~anomaly_anomalies])  # Invert the anomaly boolean for evaluation (1 = normal, 0 = anomaly)


In [35]:
# Import necessary functions from sklearn.metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)


In [36]:
# Print individual metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8790
Precision: 0.9037
Recall: 0.9500
F1 Score: 0.9263
